# Benchmark EDUC

This notebook aims to benchmark the economic dispatch and unit commitment problem.

In [1]:
import warnings
import logging
import ams
import numpy as np
import pandas as pd

import ams.benchmarks as bp

# Configure AMS logger
ams.config_logger(stream_level=20)

# Display tool versions
_ = bp.get_tool_versions()

ams.config_logger(stream_level=50)

Last run time: 2024-11-06 11:55:47
Python: 3.10.0 | packaged by conda-forge | (default, Nov 20 2021, 02:27:15) [Clang 11.1.0 ]

Tool        Version                      
----------  -----------------------------
ltbams      0.9.10.post153.dev0+gc73b7f8a
cvxpy       1.5.3                        
pandapower  2.14.11                      
PYPOWER     5.1.17                       
gurobipy    11.0.3                       
mosek       10.2.6                       
ecos        2.0.14                       
scs         3.2.7                        
piqp        0.4.2                        
numba       0.60.0                       


In [2]:
%matplotlib inline

In [3]:
ams.config_logger(stream_level=50)

In [4]:
cases = [
    './cases/pjm5bus_uced.xlsx',
    './cases/ieee14_uced.xlsx',
    './cases/ieee39_uced.xlsx',
    './cases/npcc_uced.xlsx',
    './cases/wecc_uced.xlsx',
]

*** DEBUG ***

In [5]:
# run individual case to ensure everything is working
_t, _ = bp.test_time(cases[0], routine='ED')
print(f'Time {_t}')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-25
Time [0.0007, 0.0057, 0.0061, 0.0001, 0.0001, 0.0249, 0.2706, 0.0195, 0.123]


/Users/jinningwang/work/miniconda3/envs/ams/lib/python3.10/site-packages/pandapower/create.py:5049: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  net[table] = pd.concat([net[table], dd[dd.columns[~dd.isnull().all()]]], sort=False)
/Users/jinningwang/work/miniconda3/envs/ams/lib/python3.10/site-packages/pandapower/converter/pypower/from_ppc.py:277: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  branch_lookup.loc[~is_line, "element"] = idx_trafo


In [6]:
# Suppress warnings
warnings.filterwarnings('ignore')
# show only errors
logging.getLogger('pandapower').setLevel(logging.ERROR)

# Initialize data structures for storing results
n_iters = 10
n_cases = len(cases)

time_data_ed = np.zeros((n_iters, n_cases, len(bp.cols_time)))
obj_data_ed = np.zeros((n_cases, len(bp.cols_obj)))

time_data_uc = np.zeros((n_iters, n_cases, len(bp.cols_time)))
obj_data_uc = np.zeros((n_cases, len(bp.cols_obj)))

In [7]:
# Run ED
for n_case, case in enumerate(cases):
    print(f'Case: {case}')
    for n_iter in range(n_iters):
        time, obj = bp.test_time(case)
        time_data_ed[n_iter, n_case, :] = time
    obj_data_ed[n_case, :] = obj

Case: ./cases/pjm5bus_uced.xlsx
Case: ./cases/ieee14_uced.xlsx
Case: ./cases/ieee39_uced.xlsx
Case: ./cases/npcc_uced.xlsx
Case: ./cases/wecc_uced.xlsx


In [8]:
# Run UC
for n_case, case in enumerate(cases):
    print(f'Case: {case}')
    for n_iter in range(n_iters):
        time, obj = bp.test_time(case, routine='UC')
        time_data_uc[n_iter, n_case, :] = time
    obj_data_uc[n_case, :] = obj

Case: ./cases/pjm5bus_uced.xlsx
Case: ./cases/ieee14_uced.xlsx
Case: ./cases/ieee39_uced.xlsx
Case: ./cases/npcc_uced.xlsx
Case: ./cases/wecc_uced.xlsx


In [9]:
# Process and display the results
case_names = [case.split('/')[-1].split('.')[0] for case in cases]

obj_ed = pd.DataFrame(obj_data_ed, columns=bp.cols_obj, index=case_names)

time_ed = pd.DataFrame(columns=bp.cols_time, index=case_names)
for case in case_names:
    time_ed.loc[case] = time_data_ed[:, case_names.index(case), :].mean(axis=0)

time_uc = pd.DataFrame(columns=bp.cols_time, index=case_names)
for case in case_names:
    time_uc.loc[case] = time_data_uc[:, case_names.index(case), :].mean(axis=0)

# drop the PIQP and pandapower columns
# PIQP doesn't solve UC, pandapower doesn't solve ED and UC
time_ed = time_ed.drop(columns=['ams_piqp', 'pdp'])
time_uc = time_uc.drop(columns=['ams_piqp', 'pdp'])

# scale to ms
time_ed *= 1000 
time_uc *= 1000

In [10]:
time_ed

,ams_mats,ams_parse,ams_eval,ams_final,ams_postinit,ams_grb,ams_mosek
pjm5bus_uced,0.58,1.41,2.23,0.0,0.1,5.01,5.29
ieee14_uced,0.67,1.4,2.31,0.0,0.1,5.6,5.91
ieee39_uced,0.72,1.4,2.42,0.0,0.1,5.92,6.35
npcc_uced,1.92,1.41,5.54,0.0,0.1,9.47,10.64
wecc_uced,1.63,1.44,3.85,0.0,0.1,9.45,9.45


In [11]:
time_uc

,ams_mats,ams_parse,ams_eval,ams_final,ams_postinit,ams_grb,ams_mosek
pjm5bus_uced,0.6,5.69,6.17,0.1,1.32,19.07,18.38
ieee14_uced,0.62,5.61,6.41,0.1,1.24,24.49,22.27
ieee39_uced,0.73,5.65,7.23,0.1,1.28,55.42,31.35
npcc_uced,1.5,5.66,15.9,0.1,1.92,167.94,99.19
wecc_uced,1.65,5.64,12.88,0.1,1.56,340.07,352.71


In [ ]:
time_ed.to_csv('../results/results_ed.csv')
time_uc.to_csv('../results/results_uc.csv')